# 此為最終版本

In [15]:
from gensim.models import word2vec
from pymongo import MongoClient
import random
"""
連線至MongoDB資料庫
"""
# client = MongoClient("192.168.217.135", 27017)

# 以下一行, host後面請改成已安裝docker並成功啟動思睿壓模檔案的機器IP
client = MongoClient(host='192.168.217.135', port=27017)
db_auth = client.admin
db_auth.authenticate("root","youtube")
# 連線指定資料庫，資料庫名為db_name
db = client['youtube']

"""
導入推薦系統模型
"""
# 設定欲比對詞彙
KWsearch = input('請輸入欲搜尋標籤: ')

keyword = KWsearch

topic = ['animation', 'beauty', 'entertainment.travel.daily',
         'exercise', 'food', 'game', 'knowledge', 'learn', 
         'news', 'pet', 'technology', 'variatyshow']
#載入12種Model
a = 0
TotalList=[]
maximum = 0
title = ""
checkall = 0
TopicList=[]

for i in range(12):
    try:
        model = word2vec.Word2Vec.load(topic[a] + '.word2vec.model')
        ItemList=[]
        for item in model.wv.most_similar(keyword):
            ItemList.append(item)
#             以下一行可選擇隱藏
#         print('我們在主題: ' + topic[a] + ' 搜尋到您可能喜歡的影片')
        TopicList.append(topic[a])
        a += 1
        TargetList=[]
#             以下兩行可選擇隱藏
#         print(ItemList[0][0])
#         print(ItemList[0][1])
        TargetList.append(ItemList[0][0])
        TargetList.append(ItemList[0][1])
        TotalList.append(TargetList)
        
        listorder = 0
        maximum = TotalList[listorder][1]
        for score in TotalList:
            if TotalList[listorder][1] >= maximum:
                maximum = TotalList[listorder][1]
                
                MostSimilarTopic = ""
                for y in range(len(TopicList[listorder])):
                    en = TopicList[listorder][y]
                    MostSimilarTopic += en
                
                title = ""
                for j in range(len(TotalList[listorder][0])):
                    ch = TotalList[listorder][0][j]
                    title += ch
            listorder += 1

        if a == 11:
            break
    except:
        checkall += 1
#             以下一行可選擇隱藏
#         print('我們在主題: ' + topic[a] + ' 搜尋不到您可能喜歡的影片')
        a += 1

            
if checkall ==12:
    print('\n'+'抱歉我們完全找不到您欲搜尋的影片,請再輸入一次!')
else:
#             以下五行可選擇隱藏
#     print('\n')
#     print('我們將推薦您 ' + str(MostSimilarTopic) + ' 主題')
#     print('我們認為您也喜歡 ' + str(title) + ' 的影片')
#     print('該主題與您相似值為: ' + str(maximum))
#     print('\n')
    
    MST = MostSimilarTopic
    mycol = db[str(MST)]

    import re
#     title_re = re.compile(title and keyword)
#     title_re = re.compile(keyword and title)
    title_re = re.compile(title)
    z=0
    OutPut=[]
    for x in mycol.find({'Title': title_re}, {"_id": 0, "ChannelLink": 1, 'Title': 1}):
        OutPut.append(x)
#             以下可選擇隱藏
#         print(str(z) + '. ')
#         print(x)
        z += 1
    
    x = random.randint(0,int(z))
    print('\n')
    print('我們推測您也喜歡下面頻道中這則影片: \n' + str(x) + '. ' + OutPut[int(x)]['Title'])
    print('\n')
    print('請進入該頻道搜尋: ' + 'https://www.youtube.com' + OutPut[int(x)]['ChannelLink'])

請輸入欲搜尋標籤: 大麻


我們推測您也喜歡下面頻道中這則影片: 
8. 共和黨人：我要彈劾拜登；德州要獨立？議員：還有幾個州感興趣；川普提前30年解密：將中共封在第一島鏈；印度調重軍 中共服軟撤退？8省現關聯病例 中國恐“全境淪陷”  ；驚！俄女體內現18種變異病毒；


請進入該頻道搜尋: https://www.youtube.com/channel/UC2mLqX_IavAs3TPK3GNlzAw


In [16]:
!pip install youtube-search-scraper

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for youtube-search-scraper: filename=youtube_search_scraper-1.0.0-py3-none-any.whl size=2274 sha256=d5b42566bab8aed28af229358a47feecbb0b7ade384f9b209f5357d61ff45f3e
  Stored in directory: c:\users\tibame\appdata\local\pip\cache\wheels\fc\b4\c3\aad8ced5448cb1403cbd382a92123cf48cb27eaec653aee599
Successfully built youtube-search-scraper


In [26]:
from youtube_search_scraper import *
youtube.search(keyword=OutPut[int(x)]['Title'])
response=youtube.search_results()
data=response['body']
#data=[{"viewsandtime": "220K views 2 months ago", "channel": "the quint", "title": "CBSE Exams 2021: When Will Board Exams Happen?", "link": "https://www.youtube.com/watch?v=KSg1RleLs3M"},{},...]







Progress:   0%|                                                                                               | 0/200 [00:00<?, ?it/s]






Progress:   0%|                                                                                               | 0/200 [00:00<?, ?it/s]

In [25]:
import urllib.request
import urllib.parse
import re

query_string = urllib.parse.urlencode({"search_query" : input()})
html_content = urllib.request.urlopen("http://www.youtube.com/results?" + query_string)
search_results = re.findall(r'href=\"\/watch\?v=(.{11})', html_content.read().decode())
print("http://www.youtube.com/watch?v=" + search_results[0])

共和黨人：我要彈劾拜登；德州要獨立？議員：還有幾個州感興趣；川普提前30年解密：將中共封在第一島鏈；印度調重軍 中共服軟撤退？8省現關聯病例 中國恐“全境淪陷”  ；驚！俄女體內現18種變異病毒；


IndexError: list index out of range

In [27]:
!pip install youtube-auto-search

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement youtube-auto-search (from versions: none)
ERROR: No matching distribution found for youtube-auto-search
